<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
#loading scipy library + tests
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns

In [4]:
#uploading congressional voting data file from local machine
from google.colab import files
uploaded = files.upload()

Saving house-votes-84.data to house-votes-84 (4).data


In [0]:
#creating column headers variable to pass down when I read in the csv
column_headers = ['Political Affiliation', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending', 'superfund-right-to-sue', 'crime', 'duty-free-exports', 'export-administration-act-south-africa' ]

In [7]:
#saving as dataframe variable df, reading in csv
df = pd.read_csv('house-votes-84.data', header=None, names=column_headers)

print(df.shape)
df.head()

(435, 17)


,Political Affiliation,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [20]:
#Calling the Sum of all null/NaN values by column and sorting from least to greatest 

df.isnull().sum().sort_values()

#Checking again because I *know* there are missing values
df.isnull().values.any()

False

In [21]:
#checking new column for null values 
Parties.isnull().values.any()

False

In [35]:
#Replacing all '?' values with 'Other' to more accurately convey the Values
df.replace(to_replace ="?", 
                 value ="Other")


,Political Affiliation,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,Other,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,Other
2,democrat,Other,y,y,Other,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,Other,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,Other,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,Other,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,Other,Other,Other,Other,n,y,y,y,n,y


In [0]:
#using melt to tidy dataset, changing column headers to fall under column 'Issue' + to create the Vote column
#Sorting new formatted df


In [49]:


formatted_df = pd.melt(df,
                       ["Political Affiliation"],
                       var_name="Issue",
                       value_name="Vote")
formatted_df = formatted_df.sort_values(by=["Political Affiliation"])
formatted_df.head(100)

,Political Affiliation,Issue,Vote
5991,democrat,crime,y
3217,democrat,aid-to-nicaraguan-contras,y
3215,democrat,aid-to-nicaraguan-contras,y
3214,democrat,aid-to-nicaraguan-contras,y
3213,democrat,aid-to-nicaraguan-contras,n
...,...,...,...
3317,democrat,aid-to-nicaraguan-contras,y
3316,democrat,aid-to-nicaraguan-contras,y
3306,democrat,aid-to-nicaraguan-contras,y
3315,democrat,aid-to-nicaraguan-contras,y


In [0]:
# vote = pd.concat([order_products__train, order_products__prior])


In [0]:
#combining 'other'


In [0]:
['Political Affiliation', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending', 'superfund-right-to-sue', 'crime', 'duty-free-exports', 'export-administration-act-south-africa' ]